In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Задание 1
## Цель задачи - бинарная классификация данных (сгенерированные с помощь AI или нет)

1. Загрузка данных

In [ ]:
!gdown 1lGIpg3OhOAlNPgBDTAv5AYgpIdEz2Isa # data
!gdown 1_R7mVJMgVxdlC5-TjdLnxB8HTX9unZ-_ # task

In [ ]:
!unzip 14962653.zip

In [ ]:
!unzip pan25-generative-ai-detection-task1-train.zip

In [ ]:
train_df = pd.read_json("train.jsonl", lines=True)
val_df = pd.read_json("val.jsonl", lines=True)

In [ ]:
print(f"Размер валидации: {val_df.shape[0]}")
val_df.head()

In [ ]:
print(f"Размер трейна: {train_df.shape[0]}")
train_df.head()

In [ ]:
val_df["split"] = "val"
train_df["split"] = "train"
df = pd.concat([train_df, val_df])

In [ ]:
df.head()

In [ ]:
df.isna().sum() # Данные без пропусков

посмотрим на распределение по моделям, датасетам и таргетам

In [ ]:
df["text_len"] = df["text"].str.len()

In [ ]:
columns_to_plot = ['model', 'genre', 'label']
colors = {
    0: "blue",
    1: "red"
}

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 6*len(columns_to_plot)))
fig.subplots_adjust(
    wspace=0.4,
    hspace=0.7
)

axes_flat = axes.flatten()
for i, column in enumerate(columns_to_plot):
    for i_add, split in enumerate(["train", "val"]):
      ax = axes_flat[i*2+i_add]
      # norm_val = df.loc[df["split"] == split, column].value_counts().sum()
      norm_val = 1
      ax_inner = (df.loc[df["split"] == split, column].value_counts()/norm_val).plot.bar(
          title = column + " " + split,
          ax=ax, color=colors[i_add],
          alpha=0.2,
          xlabel=""
      )
      ax_inner.set_xticklabels(ax_inner.get_xticklabels(), rotation=45, ha='right')
      ax.legend([split])

In [ ]:
for split in ["train", "val"]:

  data = df.loc[df["split"] == split, "text_len"].copy()
  data = data.clip(lower=data.quantile(0.005), upper=data.quantile(0.995))
  ax = plt.hist(data, density=True, bins = 50)
  plt.xlabel("длина текста")
  plt.title(f"длина текста {split}")
  plt.show()

In [ ]:
df.loc[df["split"] == split, "text_len"].quantile(0.98)

**Выводы из визуализации данных:**
* Не очень понятен принцип разделения на train/val;
* Есть небольшой дисбаланс классов, если будут "микро" метрики, то важно учесть;
* Распределение по типам источников данных в train/val +- совпадает. В test может отличаться;
* Оснвная масса текстов длиной до 7к символов ~= 2.5к токенов;



# Задание 2
## Цель задачи - многоклассовая классификация данных (подкатегории сгенерированных и частично сгенерированных данных + полностью написанные человеком)
## Список классов:
* Fully human-written: The document is entirely authored by a human without any AI assistance.
* Human-initiated, then machine-continued: A human starts writing, and an AI model completes the text.
* Human-written, then machine-polished: The text is initially written by a human but later refined or edited by an AI model.
* Machine-written, then machine-humanized (obfuscated): An AI generates the text, which is later modified to obscure its machine origin.
* Machine-written, then human-edited: The content is generated by an AI but subsequently edited or refined by a human.
* Deeply-mixed text: The document contains interwoven sections written by both humans and AI, without a clear separation.

1. Загрузка данных

In [ ]:
!gdown 1rNQTkhkVG9nzcT97Nk_WyJd80ZaacT0- # dev file
!gdown 1u5C4o_fmjL5nQ_RtgLDShuG97Ix6_KGK # train file

In [ ]:
val_df = pd.read_json("subtask2_dev.jsonl", lines=True)
train_df = pd.read_json("subtask2_train.jsonl", lines=True)

In [ ]:
print(f"Размер валидации: {val_df.shape[0]}")
val_df.head()

In [ ]:
print(f"Размер трейна: {train_df.shape[0]}")
train_df.head()

In [ ]:
val_df["split"] = "val"
train_df["split"] = "train"
df = pd.concat([train_df, val_df])

In [ ]:
df.head()

In [ ]:
df.isna().sum() # Данные без пропусков

посмотрим на распределение по моделям, датасетам и таргетам

In [ ]:
df["text_len"] = df["text"].str.len()

In [ ]:
columns_to_plot = ['language', 'source_dataset', 'model', 'label_text']
colors = {
    0: "blue",
    1: "red"
}

fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(12, 6*len(columns_to_plot)))
fig.subplots_adjust(
    wspace=0.4,
    hspace=0.7
)

axes_flat = axes.flatten()
for i, column in enumerate(columns_to_plot):
    for i_add, split in enumerate(["train", "val"]):
      ax = axes_flat[i*2+i_add]
      # norm_val = df.loc[df["split"] == split, column].value_counts().sum()
      norm_val = 1
      ax_inner = (df.loc[df["split"] == split, column].value_counts()/norm_val).plot.bar(
          title = column + " " + split,
          ax=ax, color=colors[i_add],
          alpha=0.2,
          xlabel=""
      )
      ax_inner.set_xticklabels(ax_inner.get_xticklabels(), rotation=45, ha='right')
      ax.legend([split])

In [ ]:
for split in ["train", "val"]:

  data = df.loc[df["split"] == split, "text_len"].copy()
  data = data.clip(lower=data.quantile(0.005), upper=data.quantile(0.995))
  ax = plt.hist(data, density=True, bins = 50)
  plt.xlabel("длина текста")
  plt.title(f"длина текста {split}")
  plt.show()

In [ ]:
df.loc[df["split"] == split, "text_len"].quantile(0.98)

**Выводы из визуализации данных:**
* Не очень понятен принцип разделения на train/val;
* Данные только на английском (хотя в задании указана мультиязычность);
* Есть сильный дисбаланс классов, если будут "микро" метрики, то важно учесть;
* Распределение по датасетам данных в train/val сильно отличается. В test скорее всего будет вообще другой набор исходных датасетов, тк они в открыом доступе (но можно проверить на такого рода лики :) );
* Сложно восстанвливать баланс, скорее всего наиболее подходящие для обучения будут metric-learning подходы;
* Оснвная масса текстов длиной до 8к символов ~= 3к токенов;
* Ссылки на статьи по используемым данным:
  - [LLM-DetectAlive](https://arxiv.org/abs/2408.04284);
  - [m4gt-bench](https://arxiv.org/abs/2402.11175) тут прям есть [данные](https://drive.google.com/drive/folders/1hBgW6sgZfz1BK0lVdUu0bZ4HPKSpOMSY);
  - [RoFT](https://arxiv.org/abs/2010.03070)
  - [MixSet](MixSet);
  - [TriBERT](https://arxiv.org/abs/2110.13412);
  - [RoFT_chatgpt](https://paperswithcode.com/dataset/roft-chatgpt) тут, кстати, гигачек тестили;
  - [Coauthor](https://arxiv.org/abs/2201.06796);
  - [LAMP](https://arxiv.org/abs/2304.11406);
* Ссылки на допонительные датасеты по тематике:
  - [датасеты с hf по детекции генеративного текста](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=trending&search=generat) тут в основном отзывы;

